In [1]:
# Imprtamos las lbrerias
import numpy as np
import pandas as pd
import json
import gzip
import ast

In [2]:
# Cremos una variable con la ruta del archivo
archivo = 'C:/Users/HP/Downloads/user_reviews.json.gz'
# Iniciamos una variable donde vamos a guarda la informacaion
data = []

# Abrimos el archivo y hacemos un for para cargar cada fila de los datos dentro de variable que definimos antes
with gzip.open(archivo, 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            json_data= ast.literal_eval(line)
            data.append(json_data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Crear un DataFrame a partir de los datos JSON
data = pd.DataFrame(data)

In [3]:
# Borramos las colunmas que no vamos a usar 
data.drop('user_url', axis=1, inplace=True)

In [4]:
# Vemos la estructura del dataframe
data.head()

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."


In [5]:
# Usamos el metodo .explote, ya que la columna de items es una coleccion, lo quenos hace el metodo explode es darnos
# Una fila del usuario por cada item en su coleccion
data_reviews = data.explode('reviews').reset_index(drop=True)

In [6]:
# Vemos la estructura del dataframe
data_reviews.head(20)

,user_id,reviews
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
5,js41637,"{'funny': '', 'posted': 'Posted November 29, 2..."
6,evcentric,"{'funny': '', 'posted': 'Posted February 3.', ..."
7,evcentric,"{'funny': '', 'posted': 'Posted December 4, 20..."
8,evcentric,"{'funny': '', 'posted': 'Posted November 3, 20..."
9,evcentric,"{'funny': '', 'posted': 'Posted October 15, 20..."


In [7]:
# Con pd.json_normalize convertimos cada lleve de las colecciones en columnas del para el dataframe
reviews = pd.json_normalize(data_reviews['reviews'])

In [8]:
# Selecionamos las variables que vamos a usar
reviews = reviews[['item_id', 'review']]

In [9]:
# Vemos el dataframe
reviews.head(10)

,item_id,review
0,1250,Simple yet with great replayability. In my opi...
1,22200,It's unique and worth a playthrough.
2,43110,Great atmosphere. The gunplay can be a bit chu...
3,251610,I know what you think when you see this title ...
4,227300,For a simple (it's actually not all that simpl...
5,239030,Very fun little game to play when your bored o...
6,248820,A suitably punishing roguelike platformer. Wi...
7,370360,"""Run for fun? What the hell kind of fun is that?"""
8,237930,"Elegant integration of gameplay, story, world ..."
9,263360,"Random drops and random quests, with stat poin..."


In [10]:
# Transformamos la columna de reviews en tipo string
reviews['review'] = reviews['review'].astype('str')

In [11]:
# Para hacer el analisis de sentimienento usamos el modulo SentimentIntensityAnalyzer de nltk.sentiment.vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Inicializamos el modulo
analisis= SentimentIntensityAnalyzer()
# Definimos una funcion que le haga un analisis de sentimientos da cada review 

def AnalisisSentimiento(text):
    sentimiento= analisis.polarity_scores(text)
    sentimiento= sentimiento['compound'] 
    # ['compound'] es un indice que te da el modulo SentimentIntensityAnalyzer  que esta entre -1 y 1 que indica 
    # Si es menor que 0 el sentimiento es negativo
    # Si es mayor que 0 pero mas cercana a 0 el sentimiento es neutral
    # si es mayor que 0 pero mas cercana a 1 es sentimiento es postivo 
    return sentimiento
    
    

In [12]:
# Con el metodo .apply le aplicamos la funcion que creamos a la columna de l os reviews creando una nueva con el resultado
reviews['SentimientoReviews']= reviews['review'].apply(AnalisisSentimiento)

In [13]:
# Vemos la columna que creamos
reviews['SentimientoReviews']

0        0.8481
1        0.2263
2        0.9117
3        0.9566
4        0.9708
          ...  
59328    0.5574
59329    0.9786
59330    0.7827
59331    0.5106
59332    0.8349
Name: SentimientoReviews, Length: 59333, dtype: float64

In [14]:
# Creamos una columna para categorizar los sentimientos, 0 si es negativo, 1 si es nuetral y 2 si es positivo
reviews['sentimiento'] = reviews['SentimientoReviews'].apply(lambda x: 0 if x < 0 else (1 if 0 <= x <= 0.3 else 2))

In [15]:
# creamos una columna individual para cada sentimiento 
reviews['senti_negativo'] = reviews['SentimientoReviews'].apply(lambda x: 1 if x < 0 else 0)
reviews['senti_neutral'] = reviews['SentimientoReviews'].apply(lambda x: 1 if  0<= x <= 0.3 else 0)
reviews['senti_positivo'] = reviews['SentimientoReviews'].apply(lambda x: 1 if x > 0.3 else 0)

In [16]:
# Mostramos el dataframe
reviews.head(10)

,item_id,review,SentimientoReviews,sentimiento,senti_negativo,senti_neutral,senti_positivo
0,1250,Simple yet with great replayability. In my opi...,0.8481,2,0,0,1
1,22200,It's unique and worth a playthrough.,0.2263,1,0,1,0
2,43110,Great atmosphere. The gunplay can be a bit chu...,0.9117,2,0,0,1
3,251610,I know what you think when you see this title ...,0.9566,2,0,0,1
4,227300,For a simple (it's actually not all that simpl...,0.9708,2,0,0,1
5,239030,Very fun little game to play when your bored o...,0.7713,2,0,0,1
6,248820,A suitably punishing roguelike platformer. Wi...,0.3658,2,0,0,1
7,370360,"""Run for fun? What the hell kind of fun is that?""",0.3313,2,0,0,1
8,237930,"Elegant integration of gameplay, story, world ...",0.4767,2,0,0,1
9,263360,"Random drops and random quests, with stat poin...",0.0000,1,0,1,0


In [17]:
# Seleccionamos las colunas que vamos a usar
reviews= reviews[['item_id','senti_negativo','senti_neutral', 'senti_positivo']]

In [18]:
# Agrupamos por id y sumamos las reviews para cada sentimiento 
reviews = reviews.groupby('item_id').agg({
    'senti_negativo': 'sum',
    'senti_neutral': 'sum',
    'senti_positivo':'sum'}).reset_index()

In [19]:
# Mostramos el resultado
reviews.head(5)

,item_id,senti_negativo,senti_neutral,senti_positivo
0,10,2,25,30
1,10090,6,7,39
2,10130,0,0,2
3,10140,1,0,0
4,10150,1,1,7


In [20]:
# Exportamos en formato csv
reviews.to_csv('C:/Users/HP/Documents/Proyectos/PI_1_Mlops/Data/Reviews.csv',index= False)

In [21]:
# Usamos el mismo dataframe del principi pra calcular las recomendaciones por usuario
users = data_reviews
users= pd.json_normalize(data_reviews['reviews'])
users['user_id']= data_reviews['user_id']
users=users[['user_id','recommend']]

In [23]:
# Tranfromamos la columna de recomend en string
users.loc[:, 'recommend'] = users['recommend'].astype('str')

In [24]:
# Aplicamos una funcion para cambiar las palabras por numeros y asi contar la cantidad de recomendaciones 
users['recommend_yes']= users['recommend'].apply(lambda x: 1 if x== 'True' else 0)
users['recommend_not']= users['recommend'].apply(lambda x: 1 if x== 'False' else 0)
users= users[['user_id', 'recommend_yes', 'recommend_not']]

In [67]:
# Agrupanos y cotamos la cantidad de recomendaciones y no reocomendaciones
users = users.groupby('user_id').agg({
    'recommend_yes': 'sum',
    'recommend_not': 'sum'}).reset_index()

In [69]:
# Exportamos el data frame en formato csv
users.to_csv('C:/Users/HP/Documents/Proyectos/PI_1_Mlops/Data/Users.csv',index= False)